# 2023 Day 08

In [34]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2023, day=8)

In [28]:



class Maps:
    def __init__(self, input_data):
        self.instructions, raw_nodes = input_data.split('\n\n')
        self.nodes = {}
        for node_label, left, right in re.findall(r'(.+) = \((.+), (.+)\)', raw_nodes):
            self.nodes[node_label] = (left, right)
        self.current_node = 'AAA'

    def navigate(self):
        for step_num, step in enumerate(itertools.cycle(self.instructions), 1):
            indx = 0 if step == 'L' else 1
            self.current_node = self.nodes[self.current_node][indx]
            if self.current_node == 'ZZZ':
                return step_num

    def __str__(self):
        node_network = '\n'.join([f'{k} = {v}' for k, v in self.nodes.items()])
        return f"Maps({self.instructions}\n\n{node_network})"

print(Maps(puzzle.examples[0].input_data).navigate())
print(Maps(puzzle.examples[1].input_data).navigate())

2
6


In [29]:
print(Maps(puzzle.input_data).navigate())

13019


Ok... this is going to get strange. I need to iterate using the steps over a bunch of nodes at once.

The large number part scares me, but I'll try to slightly modify my navigate step and my current node to be multiples and see
if it turns out ok.

In [42]:
import itertools
import re


class Maps:
    def __init__(self, input_data):
        self.logger = logging.getLogger(__name__)
        self.instructions, raw_nodes = input_data.split('\n\n')
        self.nodes = {}
        for node_label, left, right in re.findall(r'(.+) = \((.+), (.+)\)', raw_nodes):
            self.nodes[node_label] = (left, right)
        self.current_nodes = [k for k in self.nodes.keys() if k.endswith('A')]
        self.logger.debug(f'There are {len(self.current_nodes)} that will be explored')

    def navigate(self):
        for step_num, step in enumerate(itertools.cycle(self.instructions), 1):
            if step_num < 10 or step_num % 100000 == 0:
                self.logger.debug(f'Step {step_num}: {self.current_nodes}')
            indx = 0 if step == 'L' else 1

            next_nodes = []
            for current_node in self.current_nodes:
                next_nodes.append(self.nodes[current_node][indx])
            self.current_nodes = next_nodes

            if all(k.endswith('Z') for k in next_nodes):
                return step_num


    def __str__(self):
        node_network = '\n'.join([f'{k} = {v}' for k, v in self.nodes.items()])
        return f"Maps({self.instructions}\n\n{node_network})"

print(Maps(puzzle.examples[0].input_data).navigate())
print(Maps(puzzle.examples[1].input_data).navigate())
print(Maps(puzzle.examples[2].input_data).navigate())

DEBUG:aocd.models:_get_prose cache hit C:\Users\david\.config\aocd\github.davidcoe.1302241\2023_08_prose.1.html
DEBUG:aocd.examples:cached code accessors for puzzle 2023/08 part a page (23 hits)
DEBUG:aocd.examples:cached pre accessors for puzzle 2023/08 part a page (2 hits)
DEBUG:aocd.examples:cached em accessors for puzzle 2023/08 part a page (13 hits)
DEBUG:aocd.examples:cached li accessors for puzzle 2023/08 part a page (0 hits)
DEBUG:aocd.examples:cached code accessors for puzzle 2023/08 part b page (28 hits)
DEBUG:aocd.examples:cached pre accessors for puzzle 2023/08 part b page (1 hits)
DEBUG:aocd.examples:cached em accessors for puzzle 2023/08 part b page (18 hits)
DEBUG:aocd.examples:cached li accessors for puzzle 2023/08 part b page (7 hits)
DEBUG:__main__:There are 1 that will be explored
DEBUG:__main__:Step 1: ['AAA']
DEBUG:__main__:Step 2: ['CCC']
DEBUG:aocd.models:_get_prose cache hit C:\Users\david\.config\aocd\github.davidcoe.1302241\2023_08_prose.1.html
DEBUG:aocd.exam

2
6
6


In [41]:
print(f'Solution B: {Maps(puzzle.input_data).navigate()}')


DEBUG:aocd.models:input_data cache hit C:\Users\david\.config\aocd\github.davidcoe.1302241\2023_08_input.txt
DEBUG:__main__:There are 6 that will be explored
DEBUG:__main__:Step 1: ['MXA', 'VQA', 'CBA', 'JBA', 'AAA', 'HSA']
DEBUG:__main__:Step 2: ['GKS', 'TVG', 'PQX', 'HPC', 'QRB', 'HHX']
DEBUG:__main__:Step 3: ['HTQ', 'MGC', 'MXR', 'RDK', 'KHP', 'KFK']
DEBUG:__main__:Step 4: ['TNT', 'FHJ', 'HCP', 'DFN', 'LFG', 'PLP']
DEBUG:__main__:Step 5: ['BNP', 'BRJ', 'HQL', 'NVQ', 'QFF', 'JKB']
DEBUG:__main__:Step 6: ['CHM', 'NBR', 'DMC', 'NDF', 'PHC', 'XHL']
DEBUG:__main__:Step 7: ['QDC', 'TMQ', 'JKQ', 'TTF', 'RNQ', 'HGT']
DEBUG:__main__:Step 8: ['QCS', 'LFD', 'KTF', 'MMQ', 'LMG', 'BKN']
DEBUG:__main__:Step 9: ['CLJ', 'PNN', 'GBR', 'XDS', 'XHG', 'TNP']
DEBUG:__main__:Step 1000: ['KJK', 'GHN', 'BHH', 'HTF', 'KHV', 'DBM']
DEBUG:__main__:Step 2000: ['JLD', 'JCG', 'RSN', 'VFK', 'HDP', 'CXD']
DEBUG:__main__:Step 3000: ['PTC', 'RSK', 'JKQ', 'DDQ', 'SXR', 'LKS']
DEBUG:__main__:Step 4000: ['MBP', 'KKS', 

KeyboardInterrupt: 

Well, that isn't finishing quickly. Do I have a programming error or is the space
just massive? Let's put some logging in place to make sure that it's progressing as
expected....

In [39]:
import logging


logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)

Ok, looks like everything is going fine there but the number of steps must be crazy. I stopped the program at `20336000` steps.

I don't think the dictionary access is too inefficient. The list is only 6 keys so checking them all twice for the stop condition isn't an issue.

I think it's just going to take a ridicules number of cycles of each to line up.

You know in the 3rd example (the one for the 2nd part), there is a loop. They keep hitting 11Z over and over. Is that true here too?

WAIT! Before I get into that... how many "XXZ"s are there?

In [52]:
print(len(Maps(puzzle.input_data).nodes))
print([node for node in Maps(puzzle.input_data).nodes.keys() if node.endswith('Z')])
print([node for node in Maps(puzzle.input_data).nodes.keys() if node.endswith('A')])

DEBUG:aocd.models:input_data cache hit C:\Users\david\.config\aocd\github.davidcoe.1302241\2023_08_input.txt
DEBUG:__main__:There are 6 that will be explored
DEBUG:aocd.models:input_data cache hit C:\Users\david\.config\aocd\github.davidcoe.1302241\2023_08_input.txt
DEBUG:__main__:There are 6 that will be explored
DEBUG:aocd.models:input_data cache hit C:\Users\david\.config\aocd\github.davidcoe.1302241\2023_08_input.txt
DEBUG:__main__:There are 6 that will be explored


750
['QMZ', 'SHZ', 'ZZZ', 'XHZ', 'KFZ', 'FXZ']
['MXA', 'VQA', 'CBA', 'JBA', 'AAA', 'HSA']


Well, that's suspicious. The number of nodes to explore is exactly the same? I'll bet they do all have cycles and the trick is to understand when they line up.

In [ ]:
m = Maps(puzzle.examples[2].input_data)
end_nodes = [node for node in Maps(puzzle.input_data).nodes.keys() if node.endswith('Z')]

for node in end_nodes:
    print(f'Following the end node {node} around for a while...')

Ah, that won't work to look for where the end nodes go because I don't know where in the instructions to start. I'll have to go from each of the beginnings and look for a cycle instead.

In [58]:
m = Maps(puzzle.examples[2].input_data)

DEBUG:aocd.models:_get_prose cache hit C:\Users\david\.config\aocd\github.davidcoe.1302241\2023_08_prose.1.html
DEBUG:aocd.examples:cached code accessors for puzzle 2023/08 part a page (23 hits)
DEBUG:aocd.examples:cached pre accessors for puzzle 2023/08 part a page (2 hits)
DEBUG:aocd.examples:cached em accessors for puzzle 2023/08 part a page (13 hits)
DEBUG:aocd.examples:cached li accessors for puzzle 2023/08 part a page (0 hits)
DEBUG:aocd.examples:cached code accessors for puzzle 2023/08 part b page (28 hits)
DEBUG:aocd.examples:cached pre accessors for puzzle 2023/08 part b page (1 hits)
DEBUG:aocd.examples:cached em accessors for puzzle 2023/08 part b page (18 hits)
DEBUG:aocd.examples:cached li accessors for puzzle 2023/08 part b page (7 hits)
DEBUG:__main__:There are 2 that will be explored


Looking at 11A
	11A
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z
	11B
	11Z


In [61]:
start_nodes = [node for node in m.nodes.keys() if node.endswith('A')]
for node in start_nodes:
    print(f'Looking at {node}')
    for step_num, step in enumerate(itertools.cycle(m.instructions), 1):
        if step_num > 500:
            break
        indx = 0 if step == 'L' else 1
        print(f'\t{node}')
        node = m.nodes[node][indx]

Looking at MXA
	MXA
	GKS
	HTQ
	TNT
	BNP
	CHM
	QDC
	QCS
	CLJ
	XLQ
	LFQ
	XNN
	MBS
	LVN
	CXJ
	KNF
	MXN
	SBL
	FDH
	TFR
	QCD
	CVB
	JGJ
	DKM
	NDC
	DXQ
	QVH
	DPR
	TBV
	NMS
	BLK
	DKS
	VDG
	XFK
	CCR
	MNK
	FCB
	TJP
	QXF
	XXN
	SSC
	DKG
	QJT
	PVG
	LMM
	GDP
	MBP
	KCR
	VBG
	PTC
	GKJ
	HKT
	JXP
	VJT
	KVF
	JGQ
	GSK
	BCX
	NXR
	XXR
	GKS
	HTQ
	JRL
	CFT
	CHM
	QDC
	SGK
	CLJ
	KGX
	LFQ
	XNN
	MBS
	LVN
	FHS
	TPP
	XRJ
	SBL
	VVN
	TFR
	QDV
	CVB
	JGJ
	DKM
	BDM
	DXQ
	CPQ
	BLH
	TBV
	FSM
	LQH
	CRF
	RJJ
	XFK
	JFK
	KRX
	CXP
	HRN
	JBH
	TDH
	SSC
	VHJ
	QJT
	NVF
	LPT
	TFD
	MBP
	VGV
	VBG
	PTC
	LRH
	HXM
	JLD
	VJT
	DVX
	KJK
	GSK
	BVN
	KCV
	XXR
	LTM
	HTQ
	TNT
	CFT
	VXK
	QDC
	QCS
	CLJ
	KGX
	LFQ
	XNN
	MBS
	LVN
	CXJ
	KNF
	XRJ
	SBL
	VVN
	TFR
	QCD
	CHH
	JGJ
	LRR
	BDM
	TCF
	CPQ
	DPR
	GJV
	FSM
	BLK
	DKS
	RJJ
	XFK
	CCR
	MNK
	FCB
	HRN
	JBH
	XXN
	SSC
	DKG
	JNT
	NVF
	LMM
	TFD
	BJJ
	VGV
	THX
	SHL
	GKJ
	HXM
	JXP
	VJT
	KVF
	JGQ
	GSK
	BCX
	NXR
	XXR
	LTM
	HTQ
	JRL
	CFT
	VXK
	QDC
	QCS
	CLJ
	KGX
	XGP
	BCK
	MBS
	LVN
	FHS
	KNF
	MXN
	SBL
	FDH
	TFR


In [62]:
m = Maps(puzzle.input_data)

start_nodes = [node for node in m.nodes.keys() if node.endswith('A')]
for node in start_nodes:
    print(f'Looking at {node}')
    for step_num, step in enumerate(itertools.cycle(m.instructions), 1):
        if step_num > 500:
            break
        indx = 0 if step == 'L' else 1
        print(f'\t{node} {"<---" if node.endswith("Z") else ""}')
        node = m.nodes[node][indx]

DEBUG:aocd.models:input_data cache hit C:\Users\david\.config\aocd\github.davidcoe.1302241\2023_08_input.txt
DEBUG:__main__:There are 6 that will be explored


Looking at MXA
	MXA 
	GKS 
	HTQ 
	TNT 
	BNP 
	CHM 
	QDC 
	QCS 
	CLJ 
	XLQ 
	LFQ 
	XNN 
	MBS 
	LVN 
	CXJ 
	KNF 
	MXN 
	SBL 
	FDH 
	TFR 
	QCD 
	CVB 
	JGJ 
	DKM 
	NDC 
	DXQ 
	QVH 
	DPR 
	TBV 
	NMS 
	BLK 
	DKS 
	VDG 
	XFK 
	CCR 
	MNK 
	FCB 
	TJP 
	QXF 
	XXN 
	SSC 
	DKG 
	QJT 
	PVG 
	LMM 
	GDP 
	MBP 
	KCR 
	VBG 
	PTC 
	GKJ 
	HKT 
	JXP 
	VJT 
	KVF 
	JGQ 
	GSK 
	BCX 
	NXR 
	XXR 
	GKS 
	HTQ 
	JRL 
	CFT 
	CHM 
	QDC 
	SGK 
	CLJ 
	KGX 
	LFQ 
	XNN 
	MBS 
	LVN 
	FHS 
	TPP 
	XRJ 
	SBL 
	VVN 
	TFR 
	QDV 
	CVB 
	JGJ 
	DKM 
	BDM 
	DXQ 
	CPQ 
	BLH 
	TBV 
	FSM 
	LQH 
	CRF 
	RJJ 
	XFK 
	JFK 
	KRX 
	CXP 
	HRN 
	JBH 
	TDH 
	SSC 
	VHJ 
	QJT 
	NVF 
	LPT 
	TFD 
	MBP 
	VGV 
	VBG 
	PTC 
	LRH 
	HXM 
	JLD 
	VJT 
	DVX 
	KJK 
	GSK 
	BVN 
	KCV 
	XXR 
	LTM 
	HTQ 
	TNT 
	CFT 
	VXK 
	QDC 
	QCS 
	CLJ 
	KGX 
	LFQ 
	XNN 
	MBS 
	LVN 
	CXJ 
	KNF 
	XRJ 
	SBL 
	VVN 
	TFR 
	QCD 
	CHH 
	JGJ 
	LRR 
	BDM 
	TCF 
	CPQ 
	DPR 
	GJV 
	FSM 
	BLK 
	DKS 
	RJJ 
	XFK 
	CCR 
	MNK 
	FCB 
	HRN 
	JBH 
	XXN 
	SSC 
	DKG 
	JNT 
	NVF 
	LMM 
	TFD 
	

Ok... so I still think I believe that there is a cycle, but it is just absolutely insanely long? How many until I hit one ending in a `Z` for my actual inputs?

In [63]:
m = Maps(puzzle.input_data)

start_nodes = [node for node in m.nodes.keys() if node.endswith('A')]

for node in start_nodes:
    print(f'Looking at {node}')
    for step_num, step in enumerate(itertools.cycle(m.instructions), 1):
        indx = 0 if step == 'L' else 1
        node = m.nodes[node][indx]
        if node.endswith('Z'):
            print(f'Took {step_num} to reach an end')
            break

DEBUG:aocd.models:input_data cache hit C:\Users\david\.config\aocd\github.davidcoe.1302241\2023_08_input.txt
DEBUG:__main__:There are 6 that will be explored


Looking at MXA
Took 16343 to reach an end
Looking at VQA
Took 11911 to reach an end
Looking at CBA
Took 20221 to reach an end
Looking at JBA
Took 21883 to reach an end
Looking at AAA
Took 13019 to reach an end
Looking at HSA
Took 19667 to reach an end


Ok, I had the right idea, but didn't know how to solve this one. If I'd know that the least common multiple could be used to determine cycle synchronicity, I would have been good. Let's do it now.

In [66]:
import math


m = Maps(puzzle.input_data)

start_nodes = [node for node in m.nodes.keys() if node.endswith('A')]
steps = []

for node in start_nodes:
    print(f'Looking at {node}')
    for step_num, step in enumerate(itertools.cycle(m.instructions), 1):
        indx = 0 if step == 'L' else 1
        node = m.nodes[node][indx]
        if node.endswith('Z'):
            print(f'Took {step_num} to reach an end')
            steps.append(step_num)
            break

print(f'Solution B: {math.lcm(*steps)}')

DEBUG:aocd.models:input_data cache hit C:\Users\david\.config\aocd\github.davidcoe.1302241\2023_08_input.txt
DEBUG:__main__:There are 6 that will be explored


Looking at MXA
Took 16343 to reach an end
Looking at VQA
Took 11911 to reach an end
Looking at CBA
Took 20221 to reach an end
Looking at JBA
Took 21883 to reach an end
Looking at AAA
Took 13019 to reach an end
Looking at HSA
Took 19667 to reach an end
Solution B: 13524038372771
